In [1]:
!pip install -U bitsandbytes

In [2]:
# Importações essenciais
import torch
import re
import unicodedata
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model, TaskType

# Informamos qual o "cérebro" inicial que vamos usar.
# Este é um modelo já treinado pela Helsinki-NLP para traduzir de inglês para português.
model_name = "Helsinki-NLP/opus-mt-tc-big-en-pt"

In [3]:
def clean_text(text):
    """
    Função para limpar e normalizar o texto de entrada.
    """
    # Uniformiza espaços e quebras de linha
    text = text.replace("\n", " ").replace("\t", " ")
    text = re.sub(r"\s+", " ", text)
    # Remove caracteres que não são letras, números ou pontuação comum
    text = re.sub(r"[^a-zA-ZÀ-ú0-9.,;:!?()\'\" ]+", "", text)
    # Normaliza acentuação para um formato padrão
    text = unicodedata.normalize("NFKC", text)
    # Remove espaços no início e no fim
    return text.strip()

In [4]:
# O Tokenizer converte palavras em números, que é a linguagem que o modelo entende.
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Carregamos o modelo.
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    # 'device_map="auto"' usa a GPU se disponível, o que acelera muito o processo.
    device_map="auto",
    # 'load_in_8bit' é um truque para carregar o modelo usando menos memória da GPU.
    load_in_8bit=torch.cuda.is_available()
)

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/465M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [5]:
# carregamos apenas uma pequena fração (5%) do dataset para o exemplo ser rápido.
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:5%]")

In [6]:
len(dataset), dataset[0]

(14356,
 {'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box of

In [7]:
def preprocess_batch(batch):
    """
    Formata os dados de entrada e saída para o treinamento.
    """
    # foi adicionado uma instrução ("summarize:") antes de cada artigo.
    # pois, isso ensina o modelo a associar este comando com a tarefa de resumir.
    inputs = [f"summarize: {clean_text(text)}" for text in batch["article"]]
    # A saída desejada é o resumo (highlights) já limpo.
    targets = [clean_text(txt) for txt in batch["highlights"]]

    # Agora, tokenizamos (transformamos em números) tanto as entradas quanto os alvos.
    model_inputs = tokenizer(inputs, truncation=True, padding="longest")
    labels = tokenizer(targets, truncation=True, padding="longest")

    # Um detalhe técnico importante: não queremos que o modelo seja penalizado por
    # acertar os "tokens de preenchimento" (padding). Por isso, os substituímos por -100,
    # um valor que a função de cálculo de erro ignora.
    labels["input_ids"] = [
        [(token if token != tokenizer.pad_token_id else -100) for token in seq]
        for seq in labels["input_ids"]
    ]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Aplicamos a função de pré-processamento a todo o dataset.
tokenized_dataset = dataset.map(preprocess_batch, batched=True, remove_columns=dataset.column_names)

Map:   0%|          | 0/14356 [00:00<?, ? examples/s]

In [8]:
# Configuração do LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,  # Tipo de modelo que estamos usando
    r=16,                            # "Complexidade" dos adaptadores. 16 é um bom valor.
    lora_alpha=32,                   # Fator de escala. Geralmente o dobro de 'r'.
    lora_dropout=0.1,                # Ajuda a evitar que o modelo "decore" os dados.
    target_modules=["q_proj", "v_proj"] # Partes específicas do modelo que vamos adaptar.
)

# Aplicamos a configuração LoRA ao nosso modelo.
model = get_peft_model(model, lora_config)

# Veja a diferença: imprimimos a porcentagem de parâmetros que serão realmente treinados.
model.print_trainable_parameters()

trainable params: 1,179,648 || all params: 235,724,800 || trainable%: 0.5004


In [ ]:
# Argumentos que definem como o treinamento vai ocorrer.
training_args = TrainingArguments(
    output_dir="./results_summarize",      # Pasta para salvar o resultado.
    per_device_train_batch_size=4,         # Quantos exemplos processar por vez.
    learning_rate=1e-5,                    # Taxa de aprendizado (passos pequenos).
    num_train_epochs=3,                    # Quantas vezes ver o dataset inteiro.
    fp16=torch.cuda.is_available(),        # Usa precisão mista para acelerar (se tiver GPU).
    logging_steps=10,                      # Mostra o progresso a cada 10 passos.
    report_to="none"
)

# O DataCollator organiza os dados em lotes para o Trainer.
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# O Trainer é o maestro que rege todo o processo de treinamento.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# inicia o treinamento
trainer.train()

/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,8.719400
20,8.723900
30,8.351500
40,8.583400
50,8.495200
60,8.374800
70,8.358400
80,8.312900
90,8.155400
100,8.155900


/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during q

In [ ]:
# Salvamos nosso novo modelo treinado para uso futuro.
trainer.model.save_pretrained("./results_summarize")
tokenizer.save_pretrained("./results_summarize")

# Texto de exemplo para resumir
texto_artigo = """
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court.
"""

# Preparamos a entrada com o mesmo prefixo que usamos no treino.
input_text = f"summarize: {texto_artigo}"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

# Pedimos ao modelo para gerar o resumo.
output_ids = model.generate(
    **inputs,
    max_length=64,       # Definimos um tamanho máximo para o resumo.
    num_beams=4,         # 'Beam search' melhora a qualidade da resposta.
    early_stopping=True
)

# "Traduzimos" a saída numérica de volta para texto.
resposta_resumo = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("\n--- TESTE DE SUMARIZAÇÃO ---")
print(f"Texto Original (início): {texto_artigo[:150]}...")
print(f"Resumo Gerado: {resposta_resumo}")